In [1]:
from llama_index.core import ListIndex, SimpleDirectoryReader, VectorStoreIndex, Settings
from dotenv import load_dotenv
from llama_index.llms.ollama import  Ollama
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.ollama import OllamaEmbedding
import chromadb

In [2]:
Settings.llm = Ollama(model="llama3.1:latest", request_timeout=360.0, temperature=0.2, max_tokens=2048)
load_dotenv()

False

# List Index

In [11]:
documents = SimpleDirectoryReader("./data/ravens_web_official_news_10_7_10_14").load_data()
index = ListIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What's new with Baltimore Ravens?")
print(response)

KeyboardInterrupt: 

# Tree Index

In [37]:
from llama_index.core import TreeIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('./data/ravens_web_official_news_10_7_10_14').load_data()
tree_index = TreeIndex.from_documents(documents)
query_engine = tree_index.as_query_engine()
response = query_engine.query("What's new with Baltimore Ravens?")
print(response)

The Baltimore Ravens are riding a four-game winning streak, with their offense looking like a juggernaut. They have a balanced attack that is powered by running back Derrick Henry, who has been historically good this season. He has become the first player since Pro Football Hall of Famer LaDainian Tomlinson in 2005 to score a rushing touchdown in each of his first six games of a season. The Ravens are also just the second team in the Super Bowl era with at least 150 rushing yards and a rushing touchdown in each of its first six games of a season.


# Keyword Table Index

In [38]:
from llama_index.core import KeywordTableIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('./data/ravens_web_official_news_10_7_10_14').load_data()
keyword_index = KeywordTableIndex.from_documents(documents)
query_engine = keyword_index.as_query_engine()
response = query_engine.query("What's new with Baltimore Ravens?")
print(response)

**Rewrite**: The Ravens' defense has been trending upward after a physical performance against the Commanders, with several key players making bone-jarring hits and swarming to the football. This was a big game for Baltimore's defense after giving up five touchdown passes in Week 5, but they were able to raise their game by eliminating glaring mistakes and limiting Washington's longest play from scrimmage to just 28 yards. The addition of veteran coach Dean Pees has likely contributed to this improvement, as the team looks to build on its strong rush defense and shore up its secondary.


# bm25

In [1]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('./data/ravens_web_official_news_10_7_10_14').load_data()


In [2]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)

In [8]:
! pip install llama-index-retrievers-bm25

  Using cached llama_index_retrievers_bm25-0.5.2-py3-none-any.whl.metadata (740 bytes)
  Using cached bm25s-0.2.7.post1-py3-none-any.whl.metadata (21 kB)
  Using cached scipy-1.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached llama_index_retrievers_bm25-0.5.2-py3-none-any.whl (3.7 kB)
Using cached bm25s-0.2.7.post1-py3-none-any.whl (53 kB)
Using cached scipy-1.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (40.6 MB)


In [11]:
! pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.2 MB/s eta 0:00:00


In [14]:
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [15]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [16]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(bm25_retriever)

In [18]:
response = query_engine.query("What's new with Baltimore Ravens?")
print(response)

The Baltimore Ravens have unveiled three new club memberships for PSL owners at M&T Bank Stadium. These premium spaces offer unique team access, all-inclusive food and beverage options, and private restrooms. Additionally, the team is not pursuing a trade for Las Vegas Raiders wide receiver Davante Adams due to salary cap constraints.


In [20]:
! pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 61.1 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.0 MB/s eta 0:00:00


In [21]:
from llama_index.core.response.notebook_utils import display_source_node
retrieved_nodes = bm25_retriever.retrieve("What's new with Baltimore Ravens?")
for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

/home/luhj/miniconda3/envs/test/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


**Node ID:** 85b9b55a-d285-496c-be13-368f4415a466<br>**Similarity:** 1.9626601934432983<br>**Text:** Ravens Announce New Lower Concourse Club Spaces at M&T Bank Stadium
Oct 08, 2024 at 10:01 AM
Baltimore Ravens

As part of their ongoing efforts to ensure M&T Bank Stadium remains a world-class NFL and entertainment venue, the Baltimore Ravens unveiled three new club memberships to be offered to PSL owners, beginning in 2025. These premium, lower concourse club spaces serve as the latest offerings from "The Next Evolution," a three-year enhancements project created to significantly enrich the gameday and year-round fan experience at M&T Bank Stadium. (Please click the links below for more information about each new club space.)

"These new hospitality spaces bring an opportunity for PSL owners to elevate their gameday experience in areas that weren't previously possible," Ravens senior vice president of ticket sales and operations Baker Koppelman stated. "These spaces are designed with easy access in mind and provide amenities that will enhance the full-day stadium experience – from pregame to postgame. For years, we've heard from many fans who expressed a strong desire for this type of experience, so we're incredibly excited about our ability to now deliver these elevated options."

The Trust, presented by M&T Bank (2025)

The Trust, located on the south side of M&T Bank Stadium, is an ultra-premium, lower concourse club that offers unique team access, including views of the Ravens' player tunnel at the 50-yard line and the post-game press conference room. The space encapsulates a lounge-like atmosphere with design elements reminiscent of early 1900s Baltimore banks.

With club membership, PSL owners may enter The Trust beginning two hours prior to kickoff and remain up to 90 minutes following the game's conclusion. This ultra-premium club space – with a maximum capacity of 400 – offers all-inclusive premium food and beverage options and access to private restrooms.

The Raven (2025)

Situated near the 50-yard line and underneath the lower level, The Raven is located on the north side of M&T Bank Stadium and provides an elevated club experience. The space will feature thematic touches related to the team's namesake, the poem "The Raven."<br>

**Node ID:** 5f187f3d-2bdd-4d57-94f1-14b4220f4493<br>**Similarity:** 1.9011751413345337<br>**Text:** Late for Work Ravens Reportedly Not Pursuing Davante Adams
Oct 10, 2024 at 09:09 AM
Kevin Eck
Writer

Ravens Reportedly Not Pursuing Davante Adams
Speculation that Las Vegas Raiders wide receiver Davante Adams could be traded to the Ravens intensified last week when the three-time All-Pro posted a photo of Edgar Allan Poe on his Instagram along with the Poe quote, "Believe nothing you hear and half of what you see."

It remains a mystery what message Adams intended to convey with his cryptic post, but The Athletic reports that the Ravens are not among the teams pursuing him.

"Adams churned up rumors about the Baltimore Ravens' getting involved last week when he posted a picture of poet Edgar Allan Poe, who's best known for his poem 'The Raven,' on Instagram, but Baltimore is not in conversations to acquire the receiver," The Athletic's Tashan Reed, Vic Tafur, and Dianna Russini wrote.

The teams who are in talks with the Raiders about Adams are the New York Jets, New Orleans Saints, and Pittsburgh Steelers, while the Buffalo Bills are monitoring the situation, according to The Athletic.

The Ravens acquiring Adams always seemed like a longshot, largely because they are tight against the salary cap and Adams' immediate cap hit would be $13.5 million unless the Raiders ate some. Reports have suggested that Vegas has been unwilling to do so.

Lamar Jackson was asked yesterday about the Adams speculation and how satisfied he is with his pass-catchers.

"My guys are doing a great job – receivers, tight ends, running backs, fullbacks. Our guys are doing it all right now, and we're winning games, so that's all that matters," Jackson said.

Baltimore got widespread contributions from its pass catchers in the 41-38 win in Cincinnati, with Zay Flowers leading the way with seven catches for 111 yards. Rashod Bateman saw a season-high eight targets and scored a 16-yard touchdown. Even No. 3 tight end Charlie Kolar caught three passes for 64 yards and a score.

Ravens Have NFL's Best Rushing Duo
The dream team backfield of Jackson and running back Derrick Henry has lived up to the hype.<br>

# Vector Store Index

In [ ]:
# Create Chroma client
chroma_client = chromadb.EphemeralClient()

# Define collection name
collection_name = "ravens"

# Check if the collection already exists
existing_collections = chroma_client.list_collections()

if collection_name in [collection.name for collection in existing_collections]:
    chroma_collection = chroma_client.get_collection(collection_name)
    print(f"Using existing collection '{collection_name}'.")
else:
    chroma_collection = chroma_client.create_collection(collection_name)
    print(f"Created new collection '{collection_name}'.")

# Set up embedding model
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed",
    ollama_additional_kwargs={"prostatic": 0},
)

# Load documents
documents = SimpleDirectoryReader("./data/ravens_web_official_news_10_7_10_14").load_data()

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, embed_model=embed_model
# )


# with node
from llama_index.core.node_parser import SentenceSplitter, SimpleNodeParser
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model, transformations=[SimpleNodeParser(chunk_size=2048, chunk_overlap=20)]
)

Created new collection 'ravens'.


### Demo

In [23]:
# Create query engine and perform query
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query("What's new with Baltimore Ravens?")
# with open("Output.txt", "w") as text_file:
#     text_file.write(str(response))
print(response)

The Baltimore Ravens are continuing to rise in the power rankings after a thrilling overtime win against the Bengals. Lamar Jackson had an MVP showing with four touchdown passes, and the team has another tough test this week hosting the Washington Commanders. The defense is seeking answers after giving up five touchdown passes, but coordinator Zach Orr remains confident that they will find their rhythm soon.


# Hybird Retriever w/ BM25 + Chroma

## load data

In [3]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('./data/ravens_web_official_news_10_7_10_14').load_data()

## hybrid

In [35]:
from llama_index.core.node_parser import SentenceSplitter, SimpleNodeParser
hybrid_splitter = SimpleNodeParser(chunk_size=512)

hybrid_nodes = hybrid_splitter.get_nodes_from_documents(documents)

In [36]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

hybrid_docstore = SimpleDocumentStore()
hybrid_docstore.add_documents(hybrid_nodes)

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("dense_vectors")
hybrid_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

hybrid_storage_context = StorageContext.from_defaults(
    docstore=hybrid_docstore, vector_store=hybrid_vector_store
)


In [37]:
# Set up embedding model
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed",
    ollama_additional_kwargs={"prostatic": 0},
)

# vector index
from llama_index.core.node_parser import SentenceSplitter, SimpleNodeParser
hybrid_chroma_index = VectorStoreIndex.from_documents(
    documents, storage_context=hybrid_storage_context, embed_model=embed_model
)

In [38]:
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

# bm25_retriever = BM25Retriever.from_defaults(
#     docstore=docstore,
#     similarity_top_k=2,
#     # Optional: We can pass in the stemmer and set the language for stopwords
#     # This is important for removing stopwords and stemming the query + text
#     # The default is english for both
#     stemmer=Stemmer.Stemmer("english"),
#     language="english",
# )

import nest_asyncio

nest_asyncio.apply()

hybrid_retriever = QueryFusionRetriever(
    [
        hybrid_chroma_index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=hybrid_chroma_index.docstore, similarity_top_k=2
        ),
    ],
    num_queries=3,
    use_async=True,
    
    mode="reciprocal_rerank"
)


In [ ]:
# nodes = retriever.retrieve(query)
# for node in nodes:
#     display_source_node(node, source_length=5000)

NameError: name 'display_source_node' is not defined

In [39]:
from llama_index.core.query_engine import RetrieverQueryEngine

hybrid_query_engine = RetrieverQueryEngine(hybrid_retriever)

## demo

In [11]:
response = query_engine.query(query)
print(response)

The Baltimore Ravens have unveiled three new club memberships at M&T Bank Stadium, offering premium experiences for PSL owners. They also reported that they are not pursuing Las Vegas Raiders wide receiver Davante Adams due to salary cap constraints. Additionally, the team is thriving on offense with Lamar Jackson and his pass-catchers making significant contributions in their recent win against Cincinnati.


# FLARE with ChromaDB

## ChromaDB setup (copy from previous)

In [3]:
# Create Chroma client
chroma_client = chromadb.EphemeralClient()

# Define collection name
collection_name = "ravens"

# Check if the collection already exists
existing_collections = chroma_client.list_collections()

if collection_name in [collection.name for collection in existing_collections]:
    chroma_collection = chroma_client.get_collection(collection_name)
    print(f"Using existing collection '{collection_name}'.")
else:
    chroma_collection = chroma_client.create_collection(collection_name)
    print(f"Created new collection '{collection_name}'.")

# Set up embedding model
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed",
    ollama_additional_kwargs={"prostatic": 0},
)

# Load documents
documents = SimpleDirectoryReader("./data/ravens_web_official_news_10_7_10_14").load_data()

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, embed_model=embed_model
# )


# with node
from llama_index.core.node_parser import SentenceSplitter, SimpleNodeParser
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model, transformations=[SimpleNodeParser(chunk_size=512, chunk_overlap=20)]
)

Created new collection 'ravens'.


## FLARE

In [4]:
# Create query engine and perform query
index_query_engine = index.as_query_engine(similarity_top_k=2)

# create FLARE
from llama_index.core.query_engine import FLAREInstructQueryEngine

flare_query_engine = FLAREInstructQueryEngine(
    query_engine=index_query_engine,
    max_iterations=7,
    verbose=True,
)

## DEMO

In [5]:
response = index_query_engine.query("What's going on with Baltimore Ravens?")

In [6]:
print(response)

It seems like some teams in the league have been experiencing ups and downs. The Steelers' recent performance has raised concerns, while the Bengals are showing signs of improvement despite their poor record. Meanwhile, the Browns face a tough challenge due to their struggling offense.

In terms of consistency, it's worth noting that a certain team has had success when winning their division in previous years.


In [7]:
flare_response = flare_query_engine.query("What's going on with Baltimore Ravens?")

Query: What's going on with Baltimore Ravens?
Current response: 
Lookahead response: [Search(What is happening with the Baltimore Ravens?)]
Updated lookahead response: Here are the synthesized responses for each case:


Previous Response:
One of the largest cities in the world

Lookahead Template:
, the city contains a population of 
Current response:  Here are the synthesized responses for each case:


Previous Response:
One of the largest cities in the world

Lookahead Template:
, the city contains a population of
Lookahead response: [Search(What is the current status of the Baltimore Ravens football team?)]
Updated lookahead response: Here are the synthesized responses for each case:


Previous Response:
One of the largest cities in the world

Lookahead Template:
, the city contains a population of 
Current response: Here are the synthesized responses for each case:


Previous Response:
One of the largest cities in the world

Lookahead Template:
, the city contains a population of H

ReadTimeout: timed out

In [8]:
print(flare_response)

NameError: name 'flare_response' is not defined

# RAG with Qdrant

In [43]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/ravens_web_official_news_10_7_10_14").load_data()

## Split docs into small chunks

In [44]:
from llama_index.core.node_parser.text import SentenceSplitter
text_parser = SentenceSplitter(
 chunk_size=1024,
)
text_chunks = []
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

## Construct Nodes

In [45]:
from llama_index.core.schema import TextNode
nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
    text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

## Embedding Model

In [46]:
# Set up embedding model
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed",
    ollama_additional_kwargs={"prostatic": 0},
)
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
    node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [52]:
Settings.embed_model = embed_model

## Create Vector Store

In [56]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
client = qdrant_client.QdrantClient(location=":memory:")
from llama_index.core import (
 VectorStoreIndex,
 StorageContext,
 SimpleDirectoryReader,
)
vector_store = QdrantVectorStore(client=client, collection_name="my_collection")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [57]:
index = VectorStoreIndex.from_documents(
 documents, storage_context=storage_context
)

In [58]:
vector_store.add(nodes)

['82f9b08d-4c2c-4cd6-aa2e-1a139bba5105',
 '301bb0ea-7de1-455a-9283-72bb666047b9',
 '5bfec3d9-a7a2-4fab-82a1-0e811ebcaf15',
 'd2cab947-008a-4bee-9354-d91c65bc99fd',
 '529534f8-aa9f-46f9-9fd8-8b948c0b3090',
 'b6eb8758-2114-4ac4-8e6d-89856ebdf763',
 '60463f07-5294-420a-88f5-431eaef54bdc',
 'cc5b8f71-dc40-4920-80df-311a1e5383ec',
 'ea99810b-4af8-420a-9b8f-721c4a87f612',
 'd8ee931b-e5b8-4581-bf51-4458d7e2bd74',
 'acc2ce09-265e-4b84-b7a7-52a8fbe2ec48',
 'f99e2b81-10de-4136-9138-2830c907a6dc',
 'fe003cdf-9d65-4627-9705-8ba9dd98b9aa',
 '9f6612ff-c101-4ac8-9abb-1e2d4a80ed0e',
 'd68d2757-7cae-4d98-a343-9fbef6bcd9ea',
 '1e4518fb-d3f6-4039-85f6-42b84d9d0ea5',
 '0d67d06a-50d6-4a7a-921e-1bc57247560c',
 '3519df90-e624-44a5-9de5-df11baed480e',
 '86bafcba-38bd-4532-b329-ed103aebb6a6',
 '27b9ce36-0dd6-4727-b404-3c8e12e42b2f',
 '29d4da6d-c263-4c7f-8cec-6dbfcba98591',
 '427b5a35-e798-48b0-bc75-9f2911cee1e7',
 '8eee9fdc-9231-4374-9ab5-89334c5db82d',
 '45675450-355a-47d2-b6b2-dceb5729ce7b',
 'eab98441-d745-

## Query

In [59]:
query_str = "what's new with Baltimore Ravens?"
query_embedding = embed_model.get_query_embedding(query_str)

## Construct Vector Store Query

In [61]:
from llama_index.core.vector_stores import VectorStoreQuery
query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"
vector_store_query = VectorStoreQuery(
 query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

He converted big third down after big third down, did a terrific job re-discovering Mark Andrews, feeding Zay Flowers and finding guys like Tylan Wallace and Charlie Kolar when he needed them. The play to Isaiah Likely in the fourth quarter will run on highlight clips for perpetuity, and it should — but it shouldn't overshadow this gritty, brilliant performance. He looked like a two-time MVP."

The Baltimore Banner's Jonas Shaffer: "One of the first things Derrick Henry said after the game was that Lamar Jackson's the best player in the NFL. After all we saw today — the incredible highs and the very rare lows — who's going to tell him otherwise?"

The Baltimore Banner's Kyle Goon: "A long time from now, when Jackson is enshrined in Canton, they'll play the highlight of him stiff-arming Sam Hubbard to the ground, then throwing a touchdown to Isaiah Likely across his body. The two-time MVP reminded us who he is with a masterpiece in the second half, including three touchdown passes and a

## Parse the Results into a set of nodes

In [62]:
from llama_index.core.schema import NodeWithScore
from typing import Optional
nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
        nodes_with_scores.append(NodeWithScore(node=node, score=score))

## Retriever

In [64]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List

class VectorDBRetriever(BaseRetriever):
    """Retriever over a qdrant vector store."""
    def __init__(
    self,
    vector_store: QdrantVectorStore,
    embed_model: Any,
    query_mode: str = "default",
    similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()
    
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
        query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding,
        similarity_top_k=self._similarity_top_k,
        mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)
        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
                nodes_with_scores.append(NodeWithScore(node=node, score=score))
        return nodes_with_scores

In [65]:
retriever = VectorDBRetriever(
            vector_store, embed_model, query_mode="default", similarity_top_k=2
            )

## Retriever Query Engine

In [66]:
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine.from_args(
 retriever
)

## Query Results

In [67]:
query_str = "what's new with Baltimore Ravens?"
response = query_engine.query(query_str)
print(str(response))

The Baltimore Ravens have been on a roll lately. Their offense has been particularly impressive, with Lamar Jackson leading the charge. They've shown their ability to adapt and dominate games in different ways, whether it's through the air or on the ground. With players like Zay Flowers emerging as key contributors and Derrick Henry continuing to be a force to be reckoned with, this team is looking fierce.


# Random

In [7]:

from llama_index.core.memory import ChatMemoryBuffer
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions."
    ),
)

response = chat_engine.chat("Is it good for Ravens?")
print(response)

It seems like several things are going well for the Ravens from what you've shared. They had multiple players contribute in the last game, their red-zone offense is doing great with 14 of their last 15 trips resulting in touchdowns, and Justin Tucker broke his skid with a big kick despite tough conditions.


In [ ]:
response = chat_engine.chat("Is it good for Ravens?")
print(response)

# Evaluation

## prep

In [30]:
import nest_asyncio

nest_asyncio.apply()

In [15]:
! pip install wikipedia -q

In [17]:
! pip install llama-index-readers-wikipedia

In [31]:
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import VectorStoreIndex

cities = [
    "San Francisco",
]

documents = WikipediaReader().load_data(
    pages=[f"History of {x}" for x in cities]
)


In [20]:
! pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 66.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 50.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.8 MB/s eta 0:00:00
ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/home/luhj/miniconda3/envs/test/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3367, in _dep_map
    return self.__dep_map
  File "/home/luhj/miniconda3/envs/test/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3147, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistr

In [32]:
# generate questions against chunks
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

# instantiate a DatasetGenerator
dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    num_questions_per_chunk=2,  # set the number of questions per nodes
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
rag_dataset = dataset_generator.generate_dataset_from_nodes()

  0%|          | 0/14 [00:00<?, ?it/s]

  7%|▋         | 1/14 [00:14<03:10, 14.65s/it]

KeyboardInterrupt: 

100%|██████████| 2/2 [00:05<00:00,  2.64s/it]


In [34]:
from llama_index.core.evaluation import generate_question_context_pairs
qa_dataset = generate_question_context_pairs(
    hybrid_nodes,
    num_questions_per_chunk=2
)

100%|██████████| 34/34 [06:54<00:00, 12.21s/it]


In [61]:
qa_dataset

EmbeddingQAFinetuneDataset(queries={'0345c869-de41-45c2-9ad4-36beb1af539c': 'Here are two questions based on the context information:', '6bdfeb7e-a5b0-435b-9366-9cd831ae7774': '**Question 1: Multiple Choice**', '0415aba5-5226-4a8c-a1d9-e15cfc113232': 'Here are two questions based on the context information:', 'df946f7e-274f-48a5-bf76-2ce3bbb7cdbf': '**Question 1: Multiple Choice**', '9cae2a17-2706-448e-99c0-b2510506fb83': 'Here are two questions for the quiz/examination:', 'b66a6d38-db95-4a0f-91d2-7b062f2e9d99': '**Question 1: Multiple Choice**', 'ea3604eb-202f-433a-b172-4e227d97322a': 'Here are two questions based on the context information:', 'a0c707b2-d0e4-47f8-80cb-19ebd880e380': '**Question 1: Multiple Choice**', 'e80c770a-2fe0-42d3-a15c-3993e7063d6e': 'Here are two questions based on the context information:', '99df4846-2802-45f1-98a9-903b4f64ea58': '**Question 1: Factual Recall**', 'cd06c2fd-bf12-4f01-bfd5-c23a118f2eec': 'Here are two questions based on the context information:'

# EVAL CODE

In [19]:
! pip install llama-index[evaluation]

In [40]:
# from llama_index.core.llama_pack import download_llama_pack

# RagEvaluatorPack = download_llama_pack("RagEvaluatorPack", "./pack")

# rag_evaluator = RagEvaluatorPack(
#     query_engine=hybrid_query_engine,  # built with the same source Documents as the rag_dataset
#     rag_dataset=rag_dataset,
#     embed_model=embed_model,
#     judge_llm=Ollama(model="llama3.1:8b-instruct-fp16", request_timeout=360.0, temperature=0.2, max_tokens=2048)
# )
# benchmark_df = await rag_evaluator.run()

from llama_index.core.evaluation import RetrieverEvaluator

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=hybrid_retriever
)

In [41]:
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [57]:
len(eval_results)

68

In [42]:
import pandas as pd
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [44]:
display_results("LLAMA 3.1", eval_results)

,Retriever Name,Hit Rate,MRR
0,LLAMA 3.1,0.0,0.0


In [46]:
queries = list(qa_dataset.queries.values())

In [47]:
queries

['Here are two questions based on the context information:',
 '**Question 1: Multiple Choice**',
 'Here are two questions based on the context information:',
 '**Question 1: Multiple Choice**',
 'Here are two questions for the quiz/examination:',
 '**Question 1: Multiple Choice**',
 'Here are two questions based on the context information:',
 '**Question 1: Multiple Choice**',
 'Here are two questions based on the context information:',
 '**Question 1: Factual Recall**',
 'Here are two questions based on the context information:',
 '**Question 1: Multiple Choice**',
 'Here are two questions based on the context information:',
 '**Question 1: Multiple Choice**',
 'Here are two questions based on the context information:',
 '**Question 1: Multiple Choice**',
 'Here are two questions based on the context information:',
 '**Question 1: Historical Event**',
 'Here are two questions for the quiz/examination:',
 '**Question 1: Historical Event**',
 'Here are two questions based on the context

In [ ]:
from llama_index.core import ServiceContext
deepseek = Ollama(model="deepseek-r1:32b  ", temperature=0.2)
service_context_deepseek = ServiceContext.from_defaults(llm=deepseek)

ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/

In [ ]:
from llama_index.core.evaluation import FaithfulnessEvaluator
faithfulness_deepseek = FaithfulnessEvaluator(service_context=service_context_deepseek)

In [ ]:
eval_query = queries[10]

In [ ]:
response_vector = hybrid_query_engine.query(eval_query)

In [ ]:
eval_result = faithfulness_deepseek.evaluate_response(response=response_vector)

In [ ]:
eval_result.passing